In [ ]:
import marimo as mo
import nbformat

In [ ]:
from ortools.sat.python import cp_model
import math
import time
import pprint

# 極大ナップサック問題(積)

## 問題

正整数の配列 $a = [ a_1, \dots, a_n]$ と整数 $\tau$ が与えられる.
このとき $a$ の部分配列でその**総積**が $\tau$ 以下になるもので極大なものを全て列挙せよ.

## 定式化

$x = [x_1, \dots, x_n]$ を 0-1 決定変数とする.
$x_i$ が $1$ のとき $a_i$ を採用し, $0$ のとき採用しない.
また, 決定変数 $y = [y_1, \dots, y_n]$ を用意し $y_i = (a_i - 1) x_i + 1$ を課す.
こうすると $y_i$ は $x_i$ が $1$ のとき $a_i$ を値に取り, $0$ のとき $1$ を値にとる.
同様に $z_i = (a_i - 1) (1 - x_i) + 1$ とおく.
こちらは $y_i$ とは逆に採用されてない $a_i$ に対してだけ値 $a_i$ を取り, 採用されているものについては $1$ を取る.

Google OR-Tools は整数変数の積をそのまま扱えるため, 線形ではない定式化を行う.

- $\prod_{i=1}^n y_i \leq \tau$: 採用した $a_i$ の積が $\tau$ を超えない.
- $z_j \prod_{i=1}^n y_i > \tau (1 - x_j) \quad (\forall j = 1, \dots, n)$: 採用しなかった $a_j$ を掛けたら $\tau$ を超えてしまう.

また, Google OR-Tools の CP-SAT ソルバーは目的関数を設定しない場合,
実行可能解を全て求めるよう指示できるのでそれを使って全ての極大元を求める.

## 実装

In [ ]:
class SolutionPrinter(cp_model.CpSolverSolutionCallback):

    def __init__(self, a, solution: list[cp_model.IntVar]):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__solution = solution
        self.__a = a
        self.__solution_count = 0
        self.__all_solutions = []
        self.__start_time = time.time()

    @property
    def solution_count(self) -> int:
        return self.__solution_count

    @property
    def all_solutions(self):
        return self.__all_solutions

    def on_solution_callback(self):
        current_time = time.time()
        self.__solution_count = self.__solution_count + 1
        print(f'Solution {self.__solution_count}, time = {current_time - self.__start_time} s')
        print('  [', end=' ')
        for x in self.__solution:
            print(f'{self.value(x)}', end=' ')
        print(']')
        print('  [', end=' ')
        solution = []
        for i, x in enumerate(self.__solution):
            if self.value(x) == 1:
                solution.append(self.__a[i])
                print(f'{self.__a[i]}', end=' ')
        print(']')
        self.__all_solutions.append(solution)

In [ ]:
class Model:
    def __init__(self, a, tau):
        self.a = a
        self.tau = tau

        self.model = cp_model.CpModel()
        self.solver = cp_model.CpSolver()

        self.x = [self.model.new_bool_var(f"a{i} is used") for i in range(len(self.a))]
        y = [self.model.new_int_var(1, self.a[i], "") for i in range(len(self.a))]
        z = [self.model.new_int_var(1, self.a[i], "") for i in range(len(self.a))]

        for i in range(len(self.a)):
            self.model.add(y[i] == (self.a[i] - 1) * self.x[i] + 1)
            self.model.add(z[i] == (self.a[i] - 1) * (1 - self.x[i]) + 1)

        y_prod = self.model.new_int_var(1, tau, "")
        self.model.add_multiplication_equality(y_prod, y)
        for j in range(len(self.a)):
            y_prod_zj = self.model.new_int_var(1, tau * self.a[j], "")
            self.model.add_multiplication_equality(y_prod_zj, [z[j], y_prod])
            self.model.add(y_prod_zj >= self.tau * (1 - self.x[j]) + 1)

    def solve(self):
        self.solution_printer = SolutionPrinter(self.a, self.x)
        self.solver.parameters.enumerate_all_solutions = True
        #self.solver.parameters.log_search_progress = True
        self.solver.solve(self.model, self.solution_printer)

In [ ]:
# a = list(range(1, 5+1))
# tau = 10

# a = list(range(1, 10+1))
# tau = 50

a = list(range(1, 20+1))
tau = 100

# a = [2] * 16 # 少し時間かかる
# tau = 2 ** 8

print(f"a = {a}")
print(f"tau = {tau}")

a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
tau = 100


In [ ]:
model = Model(a, tau)
model.solve()

Solution 1, time = 0.006649017333984375 s
  [ 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 ]
  [ 1 2 20 ]
Solution 2, time = 0.00703883171081543 s
  [ 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 2 3 4 ]
Solution 3, time = 0.0073812007904052734 s
  [ 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 2 4 5 ]
Solution 4, time = 0.00769495964050293 s
  [ 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 2 3 5 ]
Solution 5, time = 0.008016347885131836 s
  [ 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 2 5 6 ]
Solution 6, time = 0.008352041244506836 s
  [ 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 2 3 6 ]
Solution 7, time = 0.00865316390991211 s
  [ 1 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 2 4 6 ]
Solution 8, time = 0.008961677551269531 s
  [ 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 2 6 7 ]
Solution 9, time = 0.009273052215576172 s
  [ 1 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 2 3 7 ]
Solution 10, time = 0.00959920883178711 s
  [ 1 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0

Solution 30, time = 0.01702594757080078 s
  [ 1 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 2 3 10 ]
Solution 31, time = 0.01730060577392578 s
  [ 1 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 2 4 10 ]
Solution 32, time = 0.01761913299560547 s
  [ 1 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 2 5 10 ]
Solution 33, time = 0.01793646812438965 s
  [ 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 6 10 ]
Solution 34, time = 0.018204689025878906 s
  [ 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 7 10 ]
Solution 35, time = 0.01847386360168457 s
  [ 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 8 10 ]
Solution 36, time = 0.018767356872558594 s
  [ 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 9 10 ]
Solution 37, time = 0.01905226707458496 s
  [ 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 6 9 ]
Solution 38, time = 0.019338607788085938 s
  [ 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 7 9 ]
Solution 39, time = 0.01963019371032715 s
  [ 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 

Solution 67, time = 0.0273745059967041 s
  [ 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 ]
  [ 1 5 14 ]
Solution 68, time = 0.027660369873046875 s
  [ 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 ]
  [ 1 5 13 ]
Solution 69, time = 0.027927398681640625 s
  [ 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 ]
  [ 1 4 13 ]
Solution 70, time = 0.028197765350341797 s
  [ 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 ]
  [ 1 6 13 ]
Solution 71, time = 0.028460979461669922 s
  [ 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 ]
  [ 1 7 13 ]
Solution 72, time = 0.02877664566040039 s
  [ 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 ]
  [ 1 7 12 ]
Solution 73, time = 0.029054641723632812 s
  [ 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 ]
  [ 1 6 12 ]
Solution 74, time = 0.0293123722076416 s
  [ 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 ]
  [ 1 5 12 ]
Solution 75, time = 0.029597043991088867 s
  [ 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 ]
  [ 1 8 12 ]
Solution 76, time = 0.029859542846679688 s
  [ 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 

In [ ]:
pprint.pprint(model.solution_printer.all_solutions, indent=4)

[   [1, 2, 20],
    [1, 2, 3, 4],
    [1, 2, 4, 5],
    [1, 2, 3, 5],
    [1, 2, 5, 6],
    [1, 2, 3, 6],
    [1, 2, 4, 6],
    [1, 2, 6, 7],
    [1, 2, 3, 7],
    [1, 2, 4, 7],
    [1, 2, 5, 7],
    [1, 2, 5, 8],
    [1, 2, 3, 8],
    [1, 2, 4, 8],
    [1, 2, 6, 8],
    [1, 2, 3, 9],
    [1, 2, 5, 9],
    [1, 2, 4, 9],
    [1, 2, 19],
    [1, 2, 17],
    [1, 2, 3, 16],
    [1, 2, 3, 15],
    [1, 2, 3, 14],
    [1, 2, 3, 13],
    [1, 2, 18],
    [1, 2, 3, 12],
    [1, 2, 4, 12],
    [1, 2, 4, 11],
    [1, 2, 3, 11],
    [1, 2, 3, 10],
    [1, 2, 4, 10],
    [1, 2, 5, 10],
    [1, 6, 10],
    [1, 7, 10],
    [1, 8, 10],
    [1, 9, 10],
    [1, 6, 9],
    [1, 7, 9],
    [1, 8, 9],
    [1, 3, 4, 8],
    [1, 7, 8],
    [1, 3, 4, 7],
    [1, 4, 20],
    [1, 3, 20],
    [1, 3, 4, 6],
    [1, 4, 19],
    [1, 3, 19],
    [1, 4, 16],
    [1, 6, 16],
    [1, 5, 16],
    [1, 5, 20],
    [1, 3, 4, 5],
    [1, 3, 5, 6],
    [1, 5, 19],
    [1, 5, 18],
    [1, 3, 18],
    [1, 4, 18],
    [1, 4, 17],

In [ ]:
print("Statistics")
print(f"  conflicts      : {model.solver.num_conflicts}")
print(f"  branches       : {model.solver.num_branches}")
print(f"  wall time      : {model.solver.wall_time} s")
print(f"  solutions found: {model.solution_printer.solution_count}")

Statistics
  conflicts      : 21
  branches       : 950
  wall time      : 0.030171427 s
  solutions found: 80


## 定式化(MILP)

$x = [x_1, \dots, x_n]$ を 0-1 決定変数とする.
$x_i$ が $1$ のとき $a_i$ を採用し, $0$ のとき採用しない.

$$
w := \log(a_1) x_1 + \dots + \log(a_n) x_n
$$

とすると制約条件は以下のように書ける.

- $w \leq \log(\tau)$: 採用した $a_i$ の積が $\tau$ を超えない
- $w + \log(a_i) (1 - x_i) > \log(\tau) (1 - x_i) \quad (\forall i = 1, \dots, n)$: 採用しなかった $a_j$ を掛けたら $\tau$ を超えてしまう.

CP-SAT ソルバーは整数しか扱えないので $\log(a_i)$ や $\log(\tau)$ は適当にスケールして小数点以下を切り捨てることにする.
これによって誤差が出るかもしれない.

## 実装(MILP)

In [ ]:
class ModelLinear:
    def __init__(self, a, tau):
        self.a = a
        self.tau = tau
        base = 1000000
        self.log_a = [math.floor(math.log(ai) * base) for ai in a]
        self.log_tau = math.floor(math.log(self.tau) * base)
        self.model = cp_model.CpModel()
        self.solver = cp_model.CpSolver()
        self.x = [self.model.new_bool_var(f"a{i} is used") for i in range(len(self.a))]
        w = sum(a * x for a, x in zip(self.log_a, self.x))

        self.model.add(w <= self.log_tau)
        for a, x in zip(self.log_a, self.x):
            self.model.add(w + a * (1 - x) >= (self.log_tau + 1) * (1 - x))

    def solve(self):
        self.solution_printer = SolutionPrinter(self.a, self.x)
        self.solver.parameters.enumerate_all_solutions = True
        #self.solver.parameters.log_search_progress = True
        self.solver.solve(self.model, self.solution_printer)

In [ ]:
print(f"a = {a}")
print(f"tau = {tau}")

a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
tau = 100


In [ ]:
model_linear = ModelLinear(a, tau)
model_linear.solve()

Solution 1, time = 0.0020132064819335938 s
  [ 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 3 4 5 ]
Solution 2, time = 0.002335071563720703 s
  [ 1 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 3 4 6 ]
Solution 3, time = 0.002611875534057617 s
  [ 1 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 3 5 6 ]
Solution 4, time = 0.002875089645385742 s
  [ 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 3 4 7 ]
Solution 5, time = 0.003226757049560547 s
  [ 1 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 3 4 8 ]
Solution 6, time = 0.003505706787109375 s
  [ 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 7 8 ]
Solution 7, time = 0.0037758350372314453 s
  [ 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 6 9 ]
Solution 8, time = 0.004025936126708984 s
  [ 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 7 9 ]
Solution 9, time = 0.004270076751708984 s
  [ 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 8 9 ]
Solution 10, time = 0.004540205001831055 s
  [ 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 ]

Solution 41, time = 0.012632131576538086 s
  [ 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 ]
  [ 1 4 18 ]
Solution 42, time = 0.012885808944702148 s
  [ 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 ]
  [ 1 5 18 ]
Solution 43, time = 0.013131141662597656 s
  [ 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 ]
  [ 1 3 19 ]
Solution 44, time = 0.013369560241699219 s
  [ 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 ]
  [ 1 4 19 ]
Solution 45, time = 0.013612031936645508 s
  [ 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 ]
  [ 1 5 19 ]
Solution 46, time = 0.013852357864379883 s
  [ 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 ]
  [ 1 3 20 ]
Solution 47, time = 0.014115571975708008 s
  [ 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 ]
  [ 1 4 20 ]
Solution 48, time = 0.01434946060180664 s
  [ 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 ]
  [ 1 5 20 ]
Solution 49, time = 0.014622926712036133 s
  [ 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 2 4 5 ]
Solution 50, time = 0.014883995056152344 s
  [ 1 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0

In [ ]:
pprint.pprint(model_linear.solution_printer.all_solutions, indent=4)

[   [1, 3, 4, 5],
    [1, 3, 4, 6],
    [1, 3, 5, 6],
    [1, 3, 4, 7],
    [1, 3, 4, 8],
    [1, 7, 8],
    [1, 6, 9],
    [1, 7, 9],
    [1, 8, 9],
    [1, 6, 10],
    [1, 7, 10],
    [1, 8, 10],
    [1, 9, 10],
    [1, 5, 11],
    [1, 6, 11],
    [1, 7, 11],
    [1, 8, 11],
    [1, 9, 11],
    [1, 5, 12],
    [1, 6, 12],
    [1, 7, 12],
    [1, 8, 12],
    [1, 4, 13],
    [1, 5, 13],
    [1, 6, 13],
    [1, 7, 13],
    [1, 4, 14],
    [1, 5, 14],
    [1, 6, 14],
    [1, 7, 14],
    [1, 4, 15],
    [1, 5, 15],
    [1, 6, 15],
    [1, 4, 16],
    [1, 5, 16],
    [1, 6, 16],
    [1, 3, 17],
    [1, 4, 17],
    [1, 5, 17],
    [1, 3, 18],
    [1, 4, 18],
    [1, 5, 18],
    [1, 3, 19],
    [1, 4, 19],
    [1, 5, 19],
    [1, 3, 20],
    [1, 4, 20],
    [1, 5, 20],
    [1, 2, 4, 5],
    [1, 2, 4, 6],
    [1, 2, 5, 6],
    [1, 2, 4, 7],
    [1, 2, 5, 7],
    [1, 2, 6, 7],
    [1, 2, 4, 8],
    [1, 2, 5, 8],
    [1, 2, 6, 8],
    [1, 2, 4, 9],
    [1, 2, 5, 9],
    [1, 2, 4, 10],
    [1, 2

In [ ]:
print('Statistics')
print(f' conflicts : {model_linear.solver.num_conflicts}')
print(f' branches : {model_linear.solver.num_branches}')
print(f' wall time : {model_linear.solver.wall_time} s')
print(f' solutions found: {model_linear.solution_printer.solution_count}')

Statistics
 conflicts : 14
 branches : 1104
 wall time : 0.022461598000000003 s
 solutions found: 80
